<a href="https://colab.research.google.com/github/malraharsh/Behavioral-Cloning-for-Self-Driving-Car/blob/master/Pruning_Experiments_on_DogsvsCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
! pip install -q tensorflow-model-optimization

import tempfile
import os

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
import pandas as pd


from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from tensorflow import keras
import datetime

%load_ext tensorboard

import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude, ConstantSparsity

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt



from IPython.display import display

SHOW = False

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [103]:
def makedir(path):
    os.makedirs(path, exist_ok=True)

In [105]:
log_train = 'logs/train/'
log_prune = 'logs/prune/'

makedir(log_train)
makedir(log_prune)

In [ ]:
#https://www.tensorflow.org/tutorials/images/classification

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [128]:
# train_dir = os.path.join(PATH, 'train')
# validation_dir = os.path.join(PATH, 'validation')
train_dir = os.path.join(PATH, 'validation')
validation_dir = os.path.join(PATH, 'train')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [129]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

train_image_generator = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5)

validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data


train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 1000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
def get_train_model():

    model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model

In [107]:
def train(model=None):#train_images, train_labels, test_images, test_labels):

    if model is None:
        model = get_train_model()

    log_dir = log_train + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_train, histogram_freq=1)

    history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[tensorboard_callback])
    
    return model, history.history    

In [140]:
def prune(model=None):

    if model is None:
        model = get_train_model()

    # Compute end step to finish pruning after 2 epochs.
    batch_size = 128

    # Define model for pruning.
    pruning_params = {
        'pruning_schedule': ConstantSparsity(pct_prune, 0), #target, begin step
        'block_size': (1, 1),
        'block_pooling_type': 'AVG'}

    end_step = np.ceil(total_val / batch_size).astype(np.int32) * epochs

    pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
                                                               final_sparsity=pct_prune,
                                                               begin_step=0,
                                                               end_step=end_step)}


    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
      tfmot.sparsity.keras.PruningSummaries(log_dir=log_prune),
    ]

    history = model_for_pruning.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=EPOCHS_PRUNE,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=callbacks,
    verbose=VERBOSE)

    return model_for_pruning, history.history

In [ ]:
EPOCHS = 1
EPOCHS_PRUNE = 2
VERBOSE = 1
SHOW = 0
pct_prune = 0.05

In [130]:
EPOCHS = 30
model, dt = train()

Epoch 1/30
7/7 [==============================] - 14s 2s/step - loss: 1.2434 - accuracy: 0.5183 - val_loss: 0.6971 - val_accuracy: 0.5073
Epoch 2/30
7/7 [==============================] - 14s 2s/step - loss: 0.6936 - accuracy: 0.4920 - val_loss: 0.6918 - val_accuracy: 0.5026
Epoch 3/30
7/7 [==============================] - 14s 2s/step - loss: 0.6924 - accuracy: 0.5011 - val_loss: 0.6893 - val_accuracy: 0.5000
Epoch 4/30
7/7 [==============================] - 14s 2s/step - loss: 0.6896 - accuracy: 0.4931 - val_loss: 0.6884 - val_accuracy: 0.5021
Epoch 5/30
7/7 [==============================] - 14s 2s/step - loss: 0.6855 - accuracy: 0.5034 - val_loss: 0.6795 - val_accuracy: 0.5057
Epoch 6/30
7/7 [==============================] - 14s 2s/step - loss: 0.6765 - accuracy: 0.5126 - val_loss: 0.6690 - val_accuracy: 0.5479
Epoch 7/30
7/7 [==============================] - 14s 2s/step - loss: 0.6671 - accuracy: 0.5333 - val_loss: 0.6613 - val_accuracy: 0.5708
Epoch 8/30
7/7 [==================

In [131]:
path_cp = f"training_cp-{EPOCHS:04d}.h5"
model.save(path_cp)

loaded = tf.keras.models.load_model(path_cp)

In [141]:
EPOCHS_PRUNE = 3
pct_prune = 0.05
mp, dp = prune(model)

Epoch 1/3
7/7 [==============================] - 13s 2s/step - loss: 0.6004 - accuracy: 0.6663 - val_loss: 0.5468 - val_accuracy: 0.7141
Epoch 2/3
7/7 [==============================] - 13s 2s/step - loss: 0.5791 - accuracy: 0.6709 - val_loss: 0.5677 - val_accuracy: 0.6990
Epoch 3/3
7/7 [==============================] - 13s 2s/step - loss: 0.5524 - accuracy: 0.6800 - val_loss: 0.5470 - val_accuracy: 0.7385


In [142]:
pct_prune = 0.1
mp, dp = prune(model)

Epoch 1/3
7/7 [==============================] - 13s 2s/step - loss: 0.5985 - accuracy: 0.6732 - val_loss: 0.5752 - val_accuracy: 0.6359
Epoch 2/3
7/7 [==============================] - 13s 2s/step - loss: 0.5573 - accuracy: 0.6869 - val_loss: 0.5605 - val_accuracy: 0.7276
Epoch 3/3
7/7 [==============================] - 13s 2s/step - loss: 0.5431 - accuracy: 0.7122 - val_loss: 0.5512 - val_accuracy: 0.7172


In [143]:
pct_prune = 0.2
mp, dp = prune(model)

Epoch 1/3
7/7 [==============================] - 13s 2s/step - loss: 0.6197 - accuracy: 0.6514 - val_loss: 0.5928 - val_accuracy: 0.6854
Epoch 2/3
7/7 [==============================] - 13s 2s/step - loss: 0.5815 - accuracy: 0.6640 - val_loss: 0.5756 - val_accuracy: 0.6938
Epoch 3/3
7/7 [==============================] - 13s 2s/step - loss: 0.5614 - accuracy: 0.6927 - val_loss: 0.5634 - val_accuracy: 0.7010


In [144]:
EPOCHS_PRUNE = 10
pct_prune = 0.1
mp, dp = prune(model)

Epoch 1/10
7/7 [==============================] - 13s 2s/step - loss: 0.5787 - accuracy: 0.6560 - val_loss: 0.5719 - val_accuracy: 0.6964
Epoch 2/10
7/7 [==============================] - 13s 2s/step - loss: 0.5740 - accuracy: 0.6835 - val_loss: 0.5912 - val_accuracy: 0.7323
Epoch 3/10
7/7 [==============================] - 13s 2s/step - loss: 0.5571 - accuracy: 0.6915 - val_loss: 0.5484 - val_accuracy: 0.7339
Epoch 4/10
7/7 [==============================] - 13s 2s/step - loss: 0.5515 - accuracy: 0.6972 - val_loss: 0.5282 - val_accuracy: 0.7203
Epoch 5/10
7/7 [==============================] - 13s 2s/step - loss: 0.5207 - accuracy: 0.7110 - val_loss: 0.5359 - val_accuracy: 0.7281
Epoch 6/10
7/7 [==============================] - 13s 2s/step - loss: 0.5226 - accuracy: 0.7110 - val_loss: 0.5761 - val_accuracy: 0.6646
Epoch 7/10
7/7 [==============================] - 13s 2s/step - loss: 0.5661 - accuracy: 0.6950 - val_loss: 0.5349 - val_accuracy: 0.7448
Epoch 8/10
7/7 [==================

In [ ]:
%tensorboard --logdir /content/logs/train

In [ ]:
%tensorboard --logdir /content/logs/prune